In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_enti_0 = pd.read_csv('/content/drive/MyDrive/CSC501A1/od_cntrbtn_audt_e.csv',encoding='latin-1')
df_cand_10 = pd.read_csv('/content/drive/MyDrive/CSC501A1/candidate_pre_2000_contributors_e.csv',encoding='latin-1')
df_cand_20 = pd.read_csv('/content/drive/MyDrive/CSC501A1/candidate_2000_2004_contributors_de_e.csv',encoding='latin-1')
df_poli_10 = pd.read_csv('/content/drive/MyDrive/CSC501A1/party_annual_1993-2000_contributors_e.csv', encoding='latin-1')
df_poli_20 =  pd.read_csv('/content/drive/MyDrive/CSC501A1/party_annual_2000-2004_contributors_e.csv', encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,7,8,18,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_enti = df_enti_0.copy(deep=True)
df_cand_1 = df_cand_10.copy(deep=True)
df_cand_2 = df_cand_20.copy(deep=True)
df_poli_1 = df_poli_10.copy(deep=True)
df_poli_2 = df_poli_20.copy(deep=True)

In [ ]:
# Dropping Irrelevant Columns
df_enti.drop(['Contributor last name', 'Contributor first name', 
               'Contributor middle initial','Recipient last name', 'Recipient first name', 
               'Recipient middle initial','Contribution given through'],axis=1, inplace=True)
df_cand_2.drop([' Recipient last name', ' Recipient first name', 'Contribution given through', 
                'Type of organization that contribution was given through',
                ' Recipient middle initial','Address of organization that contribution was given through']
               ,axis=1, inplace=True)
df_poli_1.drop([' Contributor last name', ' Contributor first name'],axis=1, inplace=True)
df_poli_2.drop([' Contributor last name', ' Contributor first name', 
                'Type of organization that contribution was given through',
                'Contribution given through'],axis=1, inplace=True)

# Renaming Columns to remove extra space
cols_enti = {'ï»¿Political Entity': 'Political Entity',
              'Political Party of Recipient':'Political Party',
              'Fiscal/Election date':'Fiscal date'}
cols_cand_1 = {' Contributor name': 'Contributor name'}
cols_cand_2 = {' Contributor name': 'Contributor name'}
cols_poli_1 = {' Contributor name': 'Contributor name', 'Contribution amount' : 'Monetary amount'}
cols_poli_2 = {' Contributor name': 'Contributor name'}
df_enti.rename(columns = cols_enti, inplace=True)
df_cand_1.rename(columns = cols_cand_1, inplace=True)
df_cand_2.rename(columns = cols_cand_2, inplace=True)
df_poli_1.rename(columns = cols_poli_1, inplace=True)
df_poli_2.rename(columns = cols_poli_2, inplace=True)

# Replace value to make it common
df_enti.replace({'Political Entity' : {   'ï»¿Candidates':'Candidates',
                                          'ï»¿Registered associations':'Registered associations',
                                          'ï»¿Nomination contestants':'Nomination contestants',
                                          'ï»¿Leadership contestants':'Leadership contestants',
                                          'ï»¿ï»¿Nomination contestants':'Nomination contestants',
                                          'ï»¿Registered parties':'Registered parties',
                                          'ï»¿ï»¿Registered parties':'Registered parties'},
                 'Political Party' : {
                                          'Bloc QuÃ©bÃ©cois':'Bloc Québécois',
                                          'Parti RhinocÃ©ros Party':'Parti Rhinocéros Party',
                                          'Forces et DÃ©mocratie':'Forces et Démocratie'}}, inplace=True)
df_cand_1.replace({'Political Entity' : {'Candidate':'Candidates'},
                   'Political Party' : {
                                          'Progressive Conservative Party of Canada':'Conservative Party of Canada',
                                          'Progressive Conservative':'Conservative Party of Canada',
                                          'Reform':'Reform Party of Canada',
                                          'N.D.P.':'New Democratic Party',
                                          'Independant':'Independent',
                                          'C.H.P. of Canada':'Christian Heritage Party of Canada',
                                          'Green Party':'Green Party of Canada',
                                          'Marxist-Leninist':'Marxist-Leninist Party of Canada'}}, inplace=True)
df_cand_2.replace({'Political Entity' : { 'Candidate':'Candidates'}}, inplace=True)
df_poli_1.replace({'Political Entity' : { 'Registered Party':'Registered parties'},
                   'Political Party' : {
                                          'Canadian Reform Conservative Alliance':'Conservative Party of Canada',
                                          'Progressive Conservative Party of Canada':'Conservative Party of Canada',
                                          'Liberal Party':'Liberal Party of Canada',
                                          'Progressive Conservative Party':'Conservative Party of Canada',
                                          'Reform Party':'Reform Party of Canada',
                                          'Christian Heritage Party':'Christian Heritage Party of Canada',
                                          'The Green Party of Canada':'Green Party of Canada',
                                          'Green Party':'Green Party of Canada',
                                          'Marxist-Leninist Party':'Marxist-Leninist Party of Canada'}}, inplace=True)
df_poli_2.replace({'Political Entity' : { 'Registered Party':'Registered parties'},
                   'Political Party' : {
                                          'Canadian Reform Conservative Alliance':'Conservative Party of Canada',
                                          'Progressive Conservative Party of Canada':'Conservative Party of Canada',
                                          'The Green Party of Canada':'Green Party of Canada'}}, inplace=True)

# Remove errorneous data
df_enti = df_enti[df_enti['Political Entity'] != 'ï»¿']

# Remove data where both 'Monetary amount' and 'Non-Monetary amount' is empty
df_enti.drop(df_enti[(df_enti['Monetary amount'].isnull())&(df_enti['Non-Monetary amount'].isnull())].index, inplace=True)
df_cand_1.drop(df_cand_1[(df_cand_1['Monetary amount'].isnull())&(df_cand_1['Non-Monetary amount'].isnull())].index, inplace=True)
df_cand_2.drop(df_cand_2[(df_cand_2['Monetary amount'].isnull())&(df_cand_2['Non-Monetary amount'].isnull())].index, inplace=True)
df_poli_1.drop(df_poli_1[df_poli_1['Monetary amount'].isnull()].index, inplace=True)
df_poli_2.drop(df_poli_2[(df_poli_2['Monetary amount'].isnull())&(df_poli_2['Non-Monetary amount'].isnull())].index, inplace=True)

# Remove where 'Contributor Postal code' is empty
df_enti[(df_enti['Contributor Postal code'].isnull())]

# Fill 0 where 'Non-Monetary amount' and 'Monetary Amount' is blank
cols= ['Monetary amount', 'Non-Monetary amount']
df_enti[cols]=df_enti[cols].fillna(0)
df_cand_1[cols]=df_cand_1[cols].fillna(0)
df_cand_2[cols]=df_cand_2[cols].fillna(0)
df_poli_1['Monetary amount']=df_poli_1['Monetary amount'].fillna(0)
df_poli_2[cols]=df_poli_2[cols].fillna(0)

# Remove data where any of the below column is blank
cols = ['Contributor Postal code','Contributor City','Contributor Province','Contributor name','Electoral District','Electoral event']
cols1 = ['Contributor name','Electoral District','Electoral event']
cols2 = ['Contributor name','Electoral event']
cols3 = ['Contributor Postal code','Contributor City','Contributor Province','Contributor name','Electoral event']
df_enti.drop(df_enti[df_enti[cols].isnull().any(axis = 1)].index, inplace = True)
df_cand_1.drop(df_cand_1[df_cand_1[cols1].isnull().any(axis = 1)].index, inplace = True)
df_cand_2.drop(df_cand_2[df_cand_2[cols].isnull().any(axis = 1)].index, inplace = True)
df_poli_1.drop(df_poli_1[df_poli_1[cols2].isnull().any(axis = 1)].index, inplace = True)
df_poli_2.drop(df_poli_2[df_poli_2[cols3].isnull().any(axis = 1)].index, inplace = True)

# Changing Datatype
df_enti['Recipient ID'] = df_enti['Recipient ID'].astype(int)
df_cand_1['Recipient ID'] = df_cand_1['Recipient ID'].astype(int)
df_cand_2['Recipient ID'] = df_cand_2['Recipient ID'].astype(int)
df_poli_1['Recipient ID'] = df_poli_1['Recipient ID'].astype(int)
df_poli_2['Recipient ID'] = df_poli_2['Recipient ID'].astype(int)

# df['Contributor type'].fillna(df['Contributor type'].mode()[0], inplace = True)
# df['Contributor name'].fillna(df['Contributor name'].mode()[0], inplace = True)


In [ ]:
df_enti['Contributor Province'].value_counts()

ON            277332
BC             89372
QC             77253
AB             41832
MB             24251
               ...  
NAS                1
BC'                1
Alb                1
no address         1
-                  1
Name: Contributor Province, Length: 192, dtype: int64

In [ ]:
# Creating the tables and Deleting the old instances of the tables
def db_createTableSchema():
    import sqlite3

    conn = sqlite3.connect('documents.db')
    c = conn.cursor()

    c.execute('''DROP TABLE IF EXISTS eventdetails ''')
    c.execute('''DROP TABLE IF EXISTS recipient ''')
    c.execute('''DROP TABLE IF EXISTS contributor ''')
    c.execute('''DROP TABLE IF EXISTS area ''')
    
    evnt_query = '''CREATE TABLE IF NOT EXISTS eventdetails (
                        FiscalDate DATE PRIMARY KEY NOT NULL,
                        ElectoralEvent VARCHAR(100));'''
    c.execute(evnt_query)
    print('Table eventdetails created.');
    
    recp_query = '''CREATE TABLE IF NOT EXISTS recipient (
                        RecipientID INT PRIMARY KEY NOT NULL, 
                        Recipient TEXT, 
                        PoliticalEntity VARCHAR(100),
                        PoliticalParty VARCHAR(100),
                        FiscalDate DATE,
                        FOREIGN KEY (FiscalDate) REFERENCES eventdetails(FiscalDate));'''
    c.execute(recp_query)
    print('Table recipient created.');
    
    area_query = '''CREATE TABLE IF NOT EXISTS area ( 
                        ContributorPost VARCHAR(100) PRIMARY KEY NOT NULL,
                        ContributorProv VARCHAR(100), 
                        ContributorCity VARCHAR(100));'''
    c.execute(area_query)
    print('Table area created.');
    
    cont_query = '''CREATE TABLE IF NOT EXISTS contributor (
                        ContributorName VARCHAR(100),
                        RecipientID INT, 
                        FiscalDate DATE,
                        ContributorType VARCHAR(100), 
                        ContributorPost VARCHAR(100),
                        MonAmount FLOAT,
                        NMonAmount FLOAT,
                        CONSTRAINT ContriUnq PRIMARY KEY (RecipientID,FiscalDate,ContributorName),
                        FOREIGN KEY(ContributorPost) REFERENCES area(ContributorPost),
                        FOREIGN KEY (RecipientID) REFERENCES recipient(RecipientID),
                        FOREIGN KEY (FiscalDate) REFERENCES eventdetails(FiscalDate));'''
    c.execute(cont_query)
    print('Table contributor created.');
    conn.commit()
    
# Deleting the content of the tables
def db_delete_table_content():
    import sqlite3

    conn = sqlite3.connect('documents.db')
    c = conn.cursor()

    evnt_query = """ DELETE FROM eventdetails"""
    c.execute(evnt_query)
    print("Content of table 'eventdetails' has been deleted")
    
    recp_query = """ DELETE FROM recipient"""
    c.execute(recp_query)
    print("Content of table 'recipient' has been deleted")
    
    area_query = """ DELETE FROM area"""
    c.execute(area_query)
    print("Content of table 'area' has been deleted")
    
    cont_query = """ DELETE FROM contributor"""
    c.execute(cont_query)
    print("Content of table 'contributor' has been deleted")
    conn.commit()
    
# Inserting data into the tables from files created
def db_insert_data():
    import sqlite3
    conn = sqlite3.connect('documents.db')
    c = conn.cursor()
    
    # Insert all data in tables
    for index, row in df_enti.iterrows():
        
        evnt_query = """INSERT or IGNORE INTO 
                            eventdetails(FiscalDate,ElectoralEvent) 
                            values(?,?)"""
        c.execute(evnt_query, (row['Fiscal date'], row['Electoral event']))
        
        recp_query = """INSERT or IGNORE INTO 
                            recipient(RecipientID,Recipient,PoliticalEntity,PoliticalParty,FiscalDate) 
                            values(?,?,?,?,?)"""
        c.execute(recp_query, (row['Recipient ID'], row['Recipient'], 
                               row['Political Entity'], row['Political Party'], row['Fiscal date']))
        
        area_query = """INSERT or IGNORE INTO 
                            area(ContributorPost,ContributorProv,ContributorCity) 
                            values(?,?,?)"""
        c.execute(area_query, (row['Contributor Postal code'], row['Contributor Province'], 
                               row['Contributor City']))
        
        cont_query = """INSERT or IGNORE INTO 
                            contributor(ContributorName,RecipientID,FiscalDate,ContributorType,ContributorPost,MonAmount,NMonAmount) 
                            values(?,?,?,?,?,?,?)"""
        c.execute(cont_query, (row['Contributor name'], row['Recipient ID'], 
                               row['Fiscal date'], row['Contributor type'],
                               row['Contributor Postal code'], row['Monetary amount'],
                               row['Non-Monetary amount']))
        
    for index, row in df_cand_1.iterrows():
        
        evnt_query = """INSERT or IGNORE INTO 
                            eventdetails(FiscalDate,ElectoralEvent) 
                            values(?,?)"""
        c.execute(evnt_query, (row['Fiscal date'], row['Electoral event']))
        
        recp_query = """INSERT or IGNORE INTO 
                            recipient(RecipientID,Recipient,PoliticalEntity,PoliticalParty,FiscalDate) 
                            values(?,?,?,?,?)"""
        c.execute(recp_query, (row['Recipient ID'], row['Recipient'], 
                               row['Political Entity'], row['Political Party'], row['Fiscal date']))
        
        cont_query = """INSERT or IGNORE INTO 
                            contributor(ContributorName,RecipientID,FiscalDate,ContributorType,ContributorPost,MonAmount,NMonAmount) 
                            values(?,?,?,?,?,?,?)"""
        c.execute(cont_query, (row['Contributor name'], row['Recipient ID'], 
                               row['Fiscal date'], row['Contributor type'],
                               '', row['Monetary amount'],
                               row['Non-Monetary amount']))
        
    for index, row in df_cand_2.iterrows():
        
        evnt_query = """INSERT or IGNORE INTO 
                            eventdetails(FiscalDate,ElectoralEvent) 
                            values(?,?)"""
        c.execute(evnt_query, (row['Fiscal date'], row['Electoral event']))
        
        recp_query = """INSERT or IGNORE INTO 
                            recipient(RecipientID,Recipient,PoliticalEntity,PoliticalParty,FiscalDate) 
                            values(?,?,?,?,?)"""
        c.execute(recp_query, (row['Recipient ID'], row['Recipient'], 
                               row['Political Entity'], row['Political Party'], row['Fiscal date']))
        
        area_query = """INSERT or IGNORE INTO 
                            area(ContributorPost,ContributorProv,ContributorCity) 
                            values(?,?,?)"""
        c.execute(area_query, (row['Contributor Postal code'], row['Contributor Province'], 
                               row['Contributor City']))
        
        cont_query = """INSERT or IGNORE INTO 
                            contributor(ContributorName,RecipientID,FiscalDate,ContributorType,ContributorPost,MonAmount,NMonAmount) 
                            values(?,?,?,?,?,?,?)"""
        c.execute(cont_query, (row['Contributor name'], row['Recipient ID'], 
                               row['Fiscal date'], row['Contributor type'],
                               row['Contributor Postal code'], row['Monetary amount'],
                               row['Non-Monetary amount']))
        
    for index, row in df_poli_1.iterrows():
        
        evnt_query = """INSERT or IGNORE INTO 
                            eventdetails(FiscalDate,ElectoralEvent) 
                            values(?,?)"""
        c.execute(evnt_query, (row['Fiscal date'], row['Electoral event']))
        
        recp_query = """INSERT or IGNORE INTO 
                            recipient(RecipientID,Recipient,PoliticalEntity,PoliticalParty,FiscalDate) 
                            values(?,?,?,?,?)"""
        c.execute(recp_query, (row['Recipient ID'], row['Recipient'], 
                               row['Political Entity'], row['Political Party'], row['Fiscal date']))
        
        cont_query = """INSERT or IGNORE INTO 
                            contributor(ContributorName,RecipientID,FiscalDate,ContributorType,ContributorPost,MonAmount,NMonAmount) 
                            values(?,?,?,?,?,?,?)"""
        c.execute(cont_query, (row['Contributor name'], row['Recipient ID'], 
                               row['Fiscal date'], row['Contributor type'],
                               '', row['Monetary amount'],
                               ''))
        
    for index, row in df_poli_2.iterrows():
        
        evnt_query = """INSERT or IGNORE INTO 
                            eventdetails(FiscalDate,ElectoralEvent) 
                            values(?,?)"""
        c.execute(evnt_query, (row['Fiscal date'], row['Electoral event']))
        
        recp_query = """INSERT or IGNORE INTO 
                            recipient(RecipientID,Recipient,PoliticalEntity,PoliticalParty,FiscalDate) 
                            values(?,?,?,?,?)"""
        c.execute(recp_query, (row['Recipient ID'], row['Recipient'], 
                               row['Political Entity'], row['Political Party'], row['Fiscal date']))
        
        area_query = """INSERT or IGNORE INTO 
                            area(ContributorPost,ContributorProv,ContributorCity) 
                            values(?,?,?)"""
        c.execute(area_query, (row['Contributor Postal code'], row['Contributor Province'], 
                               row['Contributor City']))
        
        cont_query = """INSERT or IGNORE INTO 
                            contributor(ContributorName,RecipientID,FiscalDate,ContributorType,ContributorPost,MonAmount,NMonAmount) 
                            values(?,?,?,?,?,?,?)"""
        c.execute(cont_query, (row['Contributor name'], row['Recipient ID'], 
                               row['Fiscal date'], row['Contributor type'],
                               row['Contributor Postal code'], row['Monetary amount'],
                               row['Non-Monetary amount']))
        
    print("Insertion into tables are completed\n")
       
    conn.commit()
    c.close()

In [ ]:
# Perform database actions
db_createTableSchema()
db_delete_table_content()
%time db_insert_data()

Table eventdetails created.
Table recipient created.
Table area created.
Table contributor created.
Content of table 'eventdetails' has been deleted
Content of table 'recipient' has been deleted
Content of table 'area' has been deleted
Content of table 'contributor' has been deleted
Insertion into tables are completed

CPU times: user 3min 17s, sys: 1.83 s, total: 3min 19s
Wall time: 3min 20s


In [ ]:
import sqlite3
conn = sqlite3.connect('documents.db',timeout=10) 
          
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM area
                               ''', conn)

In [ ]:
df = pd.DataFrame(sql_query)
print(len(df))

195251


In [ ]:
import sqlite3
conn = sqlite3.connect('documents.db',timeout=10) 
          
sql_query1 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM eventdetails
                               ''', conn)
sql_query2 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM recipient
                               ''', conn)
sql_query3 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM area
                               ''', conn)
sql_query4 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM contributor
                               ''', conn)
#print(sql_query)

df_event = pd.DataFrame(sql_query1, columns = ['FiscalDate','ElectoralEvent'])
df_recep = pd.DataFrame(sql_query2, columns = ['RecipientID','Recipient','PoliticalEntity','PoliticalParty','FiscalDate'])
df_area = pd.DataFrame(sql_query3, columns = ['ContributorPost','ContributorProv','ContributorCity'])
df_contri = pd.DataFrame(sql_query4, columns = ['ContributorName','RecipientID','FiscalDate','ContributorType','ContributorPost','MonAmount','NMonAmount'])


In [ ]:
df_event.to_csv(r'event.csv')
df_recep.to_csv(r'recep.csv')
df_area.to_csv(r'area.csv')


In [ ]:
df_contri['ContributorType'].fillna(df_contri['ContributorType'].mode()[0], inplace = True)
df_contri.to_csv(r'contri.csv')

In [ ]:
from google.colab import files
files.download('contri.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>